In [ ]:
#! pip install -U textblob
#! python -m textblob.download_corpora

In [ ]:
import pandas as pd
import re 
import seaborn as sns
import numpy as np
from textblob import TextBlob

In [ ]:
tw_clean = pd.read_csv("tw_data_short.csv")
tw_clean.info()

In [ ]:
tw_tokens = pd.read_csv("tweets_token.csv")
tw_tokens.info()

In [ ]:
all_data_token = tw_clean.join(tw_tokens, how='left', lsuffix='l', rsuffix='r', sort=False )
all_data_token["date"] = pd.to_datetime(all_data_token["date"])
all_data_token["timel"] = pd.to_datetime(all_data_token["timel"], format= '%H:%M:%S' ).dt.time
all_data_token["date_final"] = pd.to_datetime(all_data_token["date_final"])
all_data_token.rename(columns={"date_final": "time"}, inplace=True)
all_data_token

In [6]:
subj = []
polar = []
for i in range(0, len(all_data_token["tweetl"])):
    testimonial = TextBlob(all_data_token["tweetl"][i])
    subj.append(testimonial.sentiment.subjectivity)
    polar.append(testimonial.sentiment.polarity)
all_data_token["subj"] = subj
all_data_token["polar"] = polar

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
sns.scatterplot(x='polar', y='subj', hue="tweetl", data=all_data_token)
plt.xlabel('Polarity', fontsize=13)
plt.ylabel('Subjectivity', fontsize=13)
plt.title('Polarity vs Subjectivity', fontsize=15)
plt.show();

In [ ]:
all_data_token.subj.plot( kind = "hist")

In [ ]:
all_data_token.polar.plot( kind = "hist")

In [ ]:
all_data_token.drop(labels = ["hashtags", "Unnamed: 0"], axis=1, inplace=True)

In [ ]:
#Adding col with 0 1 values, wich mean more or less 100 likes
all_data_token.insert(7,'top_tw', '')
all_data_token['top_tw'] = pd.DataFrame(np.where(all_data_token['likes_count']>100, '1', '0'))
all_data_token['top_tw'].value_counts()

In [ ]:
all_data_token["top_tw"] = all_data_token["top_tw"].apply(pd.to_numeric)

In [ ]:
all_data_token.head()

In [ ]:
all_data_token.to_csv("data/all_data_pol_sub.csv")

In [ ]:
#splitting to popular and unpopular tweets
top_tw = all_data_token[all_data_token['top_tw'] == 1]
low_tw = all_data_token[all_data_token['top_tw'] == 0]
top_tw.head()

In [ ]:
all_data_token.columns

In [ ]:
cols_to_mean = ['subj', 'polar']
dict_cols = {i : "sum" if i not in cols_to_mean else "mean" for i in all_data_token.columns[3: 405]}
dict_cols

In [ ]:
#grouping by date and hour
top_tw_agg = top_tw.groupby([top_tw['time'].dt.date, top_tw['time'].dt.hour]).agg(dict_cols)
low_tw_agg = low_tw.groupby([low_tw['time'].dt.date, low_tw['time'].dt.hour]).agg(dict_cols)

In [ ]:
top_tw_agg

In [ ]:
data_prices = pd.read_csv("data/dataprices01.csv")

In [ ]:
data_prices.info()

In [ ]:
data_prices['time'] = pd.to_datetime(data_prices['time'], utc=True)
data_prices['time'] = pd.to_datetime(data_prices['time']).dt.tz_convert('Us/Mountain')
data_prices.info()

In [ ]:
data_prices = data_prices.loc[(data_prices["time"] > "2021-01-02 00:00:00") & (data_prices["time"] < "2021-04-29 01:00:00")]

In [ ]:
data_prices

In [ ]:
data_prices = data_prices.groupby([data_prices['time'].dt.date, data_prices['time'].dt.hour]).agg({"open": "sum", "high": "sum", "low": "sum", "tv": "sum", "Volume": "sum"})


In [ ]:
pd.DataFrame(data_prices)

In [ ]:
data_prices.describe()

In [ ]:
#merging data with pricess
tokens_m = top_tw_agg.merge(low_tw_agg, how = "outer",suffixes=('_t', '_l'), left_index = True, right_index = True)


In [ ]:
tokens_m.head()

In [ ]:
#merging data with pricess
all_data_m = tokens_m.merge(data_prices, how = "inner", left_index = True, right_index = True)
all_data_m.fillna(0, inplace=True)

In [ ]:
#all_data_m.to_csv("data/all_data_m.csv")

In [ ]:
all_data_m.tweetl_t[5]